# RWKV Token Shift Experiment K
This model is a custom model containing
- 24 layers
- 1024 embedding size

**Note:** This project assumes you have the rwkv-infctx conda env setup

Its primarily meant to serve as a trimming experiment of skipping stages, and reducing the dataset training requirements, to the bare-minimum required. Using the smallest known working tokenshift model.
This is believed to be possible, as the original multi stage training process was designed to work with the original RWKV v4 models.

This hopefully will help in overall speed up the experimentation process

# Basic Setup

In [ ]:
# First lets setup the various directories, and get the blank init model, these init model was generated
# using the original RWKV-LM repo (as at this point of writing, this repo cannot init a model)
# As such I have preinitialized these blank models and uploaded them to HF for convinence
!mkdir -p ../../../../model/
!mkdir -p ../../../../datapath/
!mkdir -p ../../../../checkpoint/
!cd ../../../../model/ && wget -nc https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/L24-D1024-init.pth
!ls -alh ../../../../model/L24-D1024-init.pth

# The various other stages, if you want to skip stuff

In [ ]:
DEEPSPEED_STRAT="deepspeed_stage_1"
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="TokenShift-K"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v4wavenet/"))
INFERENCE_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v4wavenet/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("INFERENCE_DIR:", INFERENCE_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

## Stage 1 : Foundation model training

In [ ]:
# Lets preload the requried dataset (enwiki_100k)
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/TokenShift-K-enwiki.yaml"

In [ ]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/TokenShift-K-enwiki.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Enwiki Foundation (ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" 

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/TokenShift-K-enwiki/last.ckpt" "../model/TokenShift-K-Stage1.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/TokenShift-K-Stage1.pth"

In [ ]:
# # Lets do a quick dragon prompt validation
!cd "{INFERENCE_DIR}" && python3 dragon_test.py ../model/TokenShift-K-Stage1.pth "cuda fp32"

In [ ]:
# Lets do a quick memory test
# (We dun expect this to work, as we have not finetune for memory recall, but its a baseline)
!python3 ../memory_script/eval_model_memory_guided.py "{PROJECT_DIR}/model/TokenShift-K-Stage1.pth"

## Tune 1 : Simple Memory instruct finetuning

- Tune 1: Low ctx size (512), Training with only the input masked. This does very limited memory training, and is used primarily to train the instruction set.

In [ ]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

# We do a strong bias for smaller word count, to teach the concept from scratch
# so that the model can learn the function. 
#
# Note that all document samples, are randomized between the target word count, 
# to half of the target word count.
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-2-count.jsonl  2  5000 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-5-count.jsonl  5  5000 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-10-count.jsonl 10 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-15-count.jsonl 15 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-20-count.jsonl 20 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-25-count.jsonl 25 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-40-count.jsonl 40 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-50-count.jsonl 50 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-60-count.jsonl 80 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-80-count.jsonl 80 2500 &

# With a slight mix of the larger word count
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-100-count.jsonl 100 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-200-count.jsonl 200 2500 &

wait
echo "## Done ##"

ls -alh ../dataset/

In [ ]:
# Lets pre tokenize the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/TokenShift-K-mem-finetune-1.yaml"

# Ensure the checkpoint directory exists
!cd "{TRAINER_DIR}" && mkdir -p "../checkpoint/TokenShift-K-mem-finetune-1/"

In [ ]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/TokenShift-K-mem-finetune-1.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Finetune-1 (bs=256, train-ctx=512, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --model.ctx_len=512

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/TokenShift-K-mem-finetune-1/last.ckpt" \
        "../model/TokenShift-K-Tune1.pth"
!cd "{TRAINER_DIR}" && ls -alh ../model/TokenShift-K-Tune1.pth

In [ ]:
# Lets do a memory eval
#
# Note that the expected performance "is not that great", as the model seems to be only loosely
# learning the memorization task, and the instruction propmt. And is seem to be acting more
# like an RNG based on the instruct. (Instead of the actual memorization task)
!python3 ../memory_script/eval_model_memory_guided.py "{PROJECT_DIR}/model/TokenShift-K-Tune1.pth"

## Tune 4 : Ramping up the ctx size (4096), memory training

- Tune 4: Mid ctx size (4096), same as tune 4, but extended in context size

This intentionally a much larger dataset, and lower learning rate to help ensure we push the model to its absolute limits.

In [ ]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

#
# Since we modify stage 4 to a single unified stage, we generate both low and high word counts
# With a slight bias towards lower word counts
# (aka 50-100 token * 2 : ~100 - 250 token ctx len)
#
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-2-count.jsonl 2 4000 &
for i in {5..45..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 3500 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 35 & 
done

#
# Ramping up the 50+ - 2000 words dataset
# 
for i in {50..2050..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 2500 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 25 & 
done

wait
echo "## Done ##"

ls -alh ../dataset/

In [ ]:
# Lets pre tokenize the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/TokenShift-K-mem-finetune-4.yaml"

# Ensure the checkpoint directory exists
!cd "{TRAINER_DIR}" && mkdir -p "../checkpoint/TokenShift-K-mem-finetune-4/"

In [ ]:
# Lets pre tokenize the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/TokenShift-K-mem-finetune-4.yaml"

# Ensure the checkpoint directory exists
!cd "{TRAINER_DIR}" && mkdir -p "../checkpoint/TokenShift-K-mem-finetune-4/"

In [ ]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/TokenShift-K-mem-finetune-4.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Finetune-4 (bs=256, train-ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --model.ctx_len=4096

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/TokenShift-K-mem-finetune-4/last.ckpt" \
        "../model/TokenShift-K-Tune4.pth"
!cd "{TRAINER_DIR}" && ls -alh ../model/TokenShift-K-Tune4.pth

In [ ]:
# Lets do a memory eval 
!python3 ../memory_script/eval_model_memory_guided.py "{PROJECT_DIR}/model/TokenShift-K-Tune4.pth"